In [1]:
import tensorflow as tf 
from fastapi import FastAPI, UploadFile, File, HTTPException
from pydantic import BaseModel
from fastapi.middleware.cors import CORSMiddleware
import io
import nest_asyncio
from predications import *

In [3]:
xry = tf.keras.models.load_model("parl.h5")
tmkc = tf.keras.models.load_model("model_tumor.h5")
alz = tf.keras.models.load_model("alzhimer_model.h5")

In [21]:
users_db = {}
app=FastAPI()
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_headers=["*"],
    allow_methods={"*"}
)

class UserSignup(BaseModel):
    username: str
    password: str

class UserLogin(BaseModel):
    username: str
    password: str


In [ ]:
@app.post("/signup/")
def signup(user: UserSignup):
    if user.username in users_db:
        raise HTTPException(status_code=400, detail="Username already exists")
    users_db[user.username] = user.password
    return {"message": "Signup successful"}

@app.post("/login/")
def login(user: UserLogin):
    if users_db.get(user.username) != user.password:
        raise HTTPException(status_code=400, detail="Invalid credentials")
    return {"message": "Login successful"}

@app.post("/parlkins")
async def analyze_x_ray(file:UploadFile=File(...)):
    contents= await file.read()
    prediction = parl(contents,xry)
    return prediction

@app.post("/tumor")
async def analyze_tumor(file:UploadFile=File(...)):
    contents= await file.read()
    prediction = tumor(contents,tmkc)
    return prediction

@app.post("/alzheimer")
async def analyze_alzheimer(file:UploadFile=File(...)):
    contents= await file.read()
    prediction = alzheimer(contents,alz)
    return prediction


In [23]:
nest_asyncio.apply()
if __name__ =="__main__":
    import uvicorn
    uvicorn.run(app,host="0.0.0.0",port=5000)

INFO:     Started server process [10064]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:5000 (Press CTRL+C to quit)


INFO:     106.193.15.53:0 - "POST /login/ HTTP/1.1" 400 Bad Request
INFO:     106.193.15.53:0 - "POST /signup/ HTTP/1.1" 200 OK
INFO:     106.193.15.53:0 - "POST /login/ HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [10064]
